<a href="https://colab.research.google.com/github/shiva-lashkari/Scalable_HW/blob/main/HW4_Scalable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 73kB/s 
     |████████████████████████████████| 204kB 49.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=629cc392e50eb0ad531453a68cd380044c5c178cd927d57cfde114bf4cafbfce
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql.session import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF 
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import DoubleType

conf = SparkConf().setMaster("local[*]").set("spark.executer.memory", "2g")

sc = SparkContext(conf=conf)
spark = SparkSession(sc).builder.getOrCreate()
spark.sparkContext.setLogLevel("ERROR")


#"review_id","user_id","business_id","stars","date","text","useful","funny","cool"

#1. Read Data
dataFrame = spark.read.format("csv").option("header", "true").option("multiline","true").load("yelp_review.csv").limit(20)
# dataFrame.printSchema()


# training = dataFrame.selectExpr("review_id as review_id", "text as text", "stars as label")
# training = dataFrame.withColumn("label", training.label.cast(DoubleType()))
training = dataFrame.withColumn("label", dataFrame.stars.cast(DoubleType()))

#2. ML pipeline
tokenizer = Tokenizer(inputCol = "text", outputCol = "words") 
TF = HashingTF(inputCol = "words", outputCol = "tfFeatures")
idf = IDF(inputCol = "tfFeatures",outputCol = "features")
lr = LogisticRegression(maxIter = 10, regParam = 0.001)

pipeline = Pipeline(stages = [tokenizer,TF,idf,lr])
model = pipeline.fit(training)

#3. Test Data
cols = [c for c in dataFrame.columns if c != "stars"]
test = dataFrame.select(cols).limit(10)
# test.printSchema()
test.show()

#4. Make Prediction
prediction = model.transform(test)

selected = prediction.select("review_id", "prediction")
count = 0
for row in selected.collect():
	id, pred = row
	count +=1
	print("Comment %s --> prediction = %f" %(count, pred))


AnalysisException: ignored